In [3]:
import numpy as np
import json
from sklearn.cluster import KMeans


# Function to read labels from JSON
def get_labels_from_json(json_file_path, resized_image_size):
    # Opening JSON file
    f = open(json_file_path)
    data = json.load(f)
    f.close()

    ori_img_height = data['height']
    ori_img_width = data['width']

    bbox_label = []
    angles = []
    for i in range(data['nb_of_plants_rows']):
        #print(data[f'row_{i}']['xmin'])
        x_min = data[f'row_{i}']['xmin']
        y_min = data[f'row_{i}']['ymin']
        x_max = data[f'row_{i}']['xmax']
        y_max = data[f'row_{i}']['ymax']
        angle = data[f'row_{i}']['angle']
        
        if x_min > x_max:
            x_min, x_max = x_max, x_min
            
        #In order to find the resized coordinates, we must multiply the ratio of the resized image compared to its original to the coordinates.
        x_min = float((resized_image_size/ori_img_width)*x_min)
        y_min = float((resized_image_size/ori_img_height)*y_min)
        x_max = float((resized_image_size/ori_img_width)*x_max)
        y_max = float((resized_image_size/ori_img_height)*y_max)

        generated_box_info = [x_min, y_min, x_max, y_max]

        #append each object's class label and the bounding box label (converted to Faster R-CNN format) into the list initialized earlier.
        bbox_label.append(np.asarray(generated_box_info, dtype='float32'))
        angles.append(angle)


    return np.asarray(bbox_label), angles



In [4]:
# Divide angles of rows in three groups depending on the cell that they belong to

bboxes = []
cells_with_angles = [[],[],[]]
for i in range(50):
    data, angles = get_labels_from_json(f'/Users/amir/Desktop/plant_row_detection/BINARY_PLANTS/labels/sample_{i}.json',300)
    bboxes.append(data)
    for j in range(len(data)):
        if data[j][0] < 100:
            cells_with_angles[0].append(angles[j])
        elif data[j][0] >= 200:
            cells_with_angles[2].append(angles[j])
        else:
            cells_with_angles[1].append(angles[j])

In [5]:
# Function to restore anchor box size from the given angle
def get_anchor_box_size(image_height, angle):
    AB = image_height - 1 
    BC = AB * np.tan(np.radians(np.abs(angle)))
    
    return (BC,image_height)

In [14]:
# Get centroids for angles of the first cell
data_2d = np.array(cells_with_angles[0]).reshape(-1, 1)
kmeans = KMeans(n_clusters=4, random_state=0).fit(data_2d)
centroids = kmeans.cluster_centers_

# Print the centroids
print("Centroids of each group:")
for i, centroid in enumerate(centroids):
    print(f"Group {i+1} centroid: {centroid[0]}")
    print(get_anchor_box_size(300,centroid[0]))


Centroids of each group:
Group 1 centroid: 13.833333333333336
(73.62591614629048, 300)
Group 2 centroid: 2.7272727272727284
(14.243125461420604, 300)
Group 3 centroid: 20.923076923076927
(114.31500698975555, 300)
Group 4 centroid: 8.111111111111114
(42.613161461881, 300)


In [15]:
# Get centroids for angles of the second cell
data_2d = np.array(cells_with_angles[1]).reshape(-1, 1)
kmeans = KMeans(n_clusters=4, random_state=0).fit(data_2d)
centroids = kmeans.cluster_centers_

# Print the centroids
print("Centroids of each group:")
for i, centroid in enumerate(centroids):
    print(f"Group {i+1} centroid: {centroid[0]}")
    print(get_anchor_box_size(300,centroid[0]))

Centroids of each group:
Group 1 centroid: -2.145833333333332
(11.203343784676777, 300)
Group 2 centroid: -17.90909090909091
(96.62676173785495, 300)
Group 3 centroid: -10.6875
(56.42907866051493, 300)
Group 4 centroid: 4.363636363636366
(22.815917012095873, 300)


In [16]:
# Get centroids for angles of the third cell
data_2d = np.array(cells_with_angles[2]).reshape(-1, 1)
kmeans = KMeans(n_clusters=4, random_state=0).fit(data_2d)
centroids = kmeans.cluster_centers_

# Print the centroids
print("Centroids of each group:")
for i, centroid in enumerate(centroids):
    print(f"Group {i+1} centroid: {centroid[0]}")
    print(get_anchor_box_size(300,centroid[0]))

Centroids of each group:
Group 1 centroid: -10.583333333333334
(55.86631221021648, 300)
Group 2 centroid: -5.833333333333334
(30.547068787658652, 300)
Group 3 centroid: -14.25
(75.93632629600816, 300)
Group 4 centroid: -1.0
(5.219064413537058, 300)


### Score to evaluate the model (Overlap Score and Fitness score)

In [4]:
import cv2 as cv
import numpy as np

In [107]:
def get_bbox_values(image_height, image_width, xmin, ymin, xmax, ymax, plant_size):
    img = np.zeros((image_height, image_width), np.uint8)
    cv.line(img, (xmin, ymin), (xmax, ymax), 255, plant_size)
    bbox_values = np.where(img != 0)
    bbox_coordinates = np.array(list(zip(bbox_values[1], bbox_values[0])))
    return bbox_coordinates


def get_angle(xmin,ymin,xmax,ymax):
    a = np.array([xmin, ymax])
    b = np.array([xmin, ymin])
    c = np.array([xmax, ymax])
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    if xmin > xmax:
        return -np.degrees(angle)
    else:
        return np.degrees(angle)

In [164]:
def overlap_score(bbox_gt_coordinates, bbox_pred_coordinates):
  len_intersection = len(np.array([x for x in set(tuple(x) for x in bbox_gt_coordinates) & set(tuple(x) for x in bbox_pred_coordinates)]))
  return len_intersection / len(bbox_gt_coordinates)

def fitness_score(angle_gt, angle_pred):
  return 1 - (np.abs(angle_gt - angle_pred) / 180)

def composite_score(alpha, overlap_score, fitness_score):
  return alpha * overlap_score + (1 - alpha) * fitness_score

In [155]:
image_height = 300
image_width = 300
xmin = 100
ymin = 0
xmax = 100
ymax = 300
plant_size = 25

In [165]:
bbox_gt_coordinates = get_bbox_values(image_height, image_width, xmin, ymin, xmax, ymax, plant_size)
bbox_pred_coordinates = get_bbox_values(300, 300, 101, 0, 101, 300, 25)

angle_gt = get_angle(xmin, ymin, xmax, ymax)
angle_pred = get_angle(101, 0, 101, 300)

In [166]:
o_score = overlap_score(bbox_gt_coordinates, bbox_pred_coordinates)
print(f"Overlap score: {o_score}")

f_score = fitness_score(angle_gt, angle_pred)
print(f"Fitness score: {f_score}")

comp_score = composite_score(0.5, o_score, f_score)
print(f"Composite score score: {comp_score}")

Overlap score: 0.9629629629629629
Fitness score: 1.0
Composite score score: 0.9814814814814814
